In [24]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt

In [3]:
import os
os.listdir()

['.DS_Store',
 'CrimeAnalysis.ipynb',
 '.ipynb_checkpoints',
 'chi-crime-analysis.py',
 'raw']

In [65]:
crime17 = pd.read_csv('raw/Crimes-2017.csv')
crime18 = pd.read_csv('raw/Crimes-2018.csv')
comm_areas = gpd.read_file('geo/CHI_comm_areas.geojson')

all_crimes = pd.concat([crime17, crime18])
all_crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11094370,JA440032,09/21/2017 12:15:00 AM,072XX N CALIFORNIA AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,CURRENCY EXCHANGE,True,False,...,50.0,2.0,10,1156443.0,1947707.0,2017,03/01/2018 03:52:35 PM,42.012293,-87.699714,"(42.012293397, -87.699714109)"
1,11118031,JA470589,10/12/2017 07:14:00 PM,055XX W GRAND AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,JAIL / LOCK-UP FACILITY,True,False,...,29.0,19.0,14,1138788.0,1913480.0,2017,03/01/2018 03:52:35 PM,41.918712,-87.765511,"(41.918711651, -87.76551063)"
2,11134189,JA491697,10/30/2017 11:52:00 AM,043XX S TALMAN AVE,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,APARTMENT,True,False,...,12.0,58.0,26,1159425.0,1875711.0,2017,03/01/2018 03:52:35 PM,41.814670,-87.690727,"(41.814669784, -87.690726696)"
3,11156462,JA521389,09/29/2017 06:45:00 PM,055XX W BELMONT AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,CURRENCY EXCHANGE,True,False,...,30.0,19.0,11,1138653.0,1920720.0,2017,03/01/2018 03:52:35 PM,41.938581,-87.765831,"(41.938581442, -87.765830579)"
4,11164874,JA531910,12/01/2017 06:20:00 AM,022XX W CHICAGO AVE,0265,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER,STREET,True,False,...,32.0,24.0,02,1161264.0,1905292.0,2017,03/01/2018 03:52:35 PM,41.895805,-87.683159,"(41.895805099, -87.683159355)"


In [60]:
#comm_areas.head()
comm_xwalk = comm_areas[['community', 'area_num_1']]

,community,area_num_1
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39
5,LINCOLN SQUARE,4
6,WASHINGTON PARK,40
7,HYDE PARK,41
8,WOODLAWN,42
9,ROGERS PARK,1


In [21]:
types17 = crime17[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
types18 = crime18[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
type_summary = types17.join(types18, on='Primary Type', how='outer', lsuffix = '_2017', rsuffix='_2018')
type_summary

,ID_2017,ID_2018
Primary Type,,
THEFT,64345,65079
BATTERY,49214,49781
CRIMINAL DAMAGE,29042,27806
ASSAULT,19303,20377
DECEPTIVE PRACTICE,19025,18716
OTHER OFFENSE,17227,17125
BURGLARY,13001,11729
ROBBERY,11877,9683
NARCOTICS,11658,12987


In [37]:
nhoods17 = crime17[['Community Area', 'ID']].groupby(['Community Area']).count().sort_values(by='ID', ascending=False)
nhoods18 = crime18[['Community Area', 'ID']].groupby(['Community Area']).count().sort_values(by='ID', ascending=False)
nhood_summary = nhoods17.join(nhoods18, on='Community Area', how='outer', lsuffix = '_2017', rsuffix='_2018')
nhood_summary
#pd.merge(nhood_summary, comm_areas[['community', 'area_num_1']], left_on='Community Area', right_on='area_num_1')

,ID_2017,ID_2018
Community Area,,
25,15385,15041
8,12311,13056
32,10624,10856
29,9066,9310
28,9049,9399
43,8677,8648
24,8324,7301
23,8055,8039
71,7527,7370


In [77]:
nhoods_crime = all_crimes[['Community Area', 'Primary Type', 'ID']].groupby(['Community Area', 'Primary Type']).count().sort_values(by='ID', ascending=False).reset_index()
nhoods_crime.columns = ['area_num_1', 'Crime Type', 'Crimes']
nhoods_crime.groupby(['area_num_1']).max().reset_index()[['area_num_1', 'Crime Type', 'Crimes']]

,area_num_1,Crime Type,Crimes
0,0.0,NARCOTICS,1
1,1.0,WEAPONS VIOLATION,2282
2,2.0,WEAPONS VIOLATION,1725
3,3.0,WEAPONS VIOLATION,1954
4,4.0,WEAPONS VIOLATION,1170
5,5.0,WEAPONS VIOLATION,1057
6,6.0,WEAPONS VIOLATION,4581
7,7.0,WEAPONS VIOLATION,4735
8,8.0,WEAPONS VIOLATION,12023
9,9.0,WEAPONS VIOLATION,115


In [58]:
#nhood_crime_summary.merge(comm_areas[['community', 'area_num_1']], on='area_num_1')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [101]:
arrests = all_crimes[['Primary Type', 'Arrest', 'ID']].groupby(['Primary Type', 'Arrest']).count().sort_values(by='Primary Type', ascending=False).reset_index()
arrests = arrests.pivot(index='Primary Type', columns='Arrest', values='ID').reset_index()
arrests.columns = ['Primary Type', 'NoArrest', 'Arrest']
arrests['Arrest Rate'] = arrests['Arrest'] / (arrests['Arrest'] + arrests['NoArrest'])
arrests[['Primary Type', 'Arrest Rate']].sort_values(by='Arrest Rate', ascending=False)

,Primary Type,Arrest Rate
17,NARCOTICS,0.999067
4,CONCEALED CARRY LICENSE VIOLATION,0.967890
12,INTERFERENCE WITH PUBLIC OFFICER,0.951903
20,OBSCENITY,0.797688
31,WEAPONS VIOLATION,0.737569
26,PUBLIC PEACE VIOLATION,0.644227
22,OTHER NARCOTIC VIOLATION,0.583333
7,CRIMINAL TRESPASS,0.570064
19,NON-CRIMINAL (SUBJECT SPECIFIED),0.400000
10,HOMICIDE,0.258320
